In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.metrics import accuracy_score

# Figures inline and set visualization style
%matplotlib inline
sns.set()

In [ ]:
df_train = pd.read_csv("/kaggle/input/titanic/train.csv")
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
survived_train=df_train.Survived
data=pd.concat([df_train.drop(['Survived'],axis=1),df_test],sort=False)

In [ ]:
data['Age']=data.Age.fillna(data.Age.median())
data['Fare']=data.Fare.fillna(data.Fare.median())

In [ ]:
def Minor_Encoder(row):
    if row["Age"] < 16:
        return 1
    else:
        return 0

data['Minor']  = data.apply (lambda row: Minor_Encoder(row),axis=1)


In [ ]:
data=data.drop(['Embarked'],axis=1)

In [ ]:
data=data.drop(['Ticket'],axis=1)
data.tail()

In [ ]:
data['FamilySize']=data['Parch']+data['SibSp']

In [ ]:
data=data.drop(['SibSp','Parch'],axis=1)

In [ ]:
data.head()

In [ ]:
data_dum= pd.get_dummies(data,prefix="Sex",columns = ["Sex"])
data_dum=data_dum.drop(['Sex_female'],axis=1)
data_dum=data_dum.drop(['PassengerId','Cabin'],axis=1)
data_dum.head()

In [ ]:
data_dum = pd.get_dummies(data_dum,prefix="Pclass",columns = ["Pclass"],drop_first=True)

In [ ]:
firstname=[]
for row in data_dum["Name"]:
        surname=row.split(",")[0]
        firstname.append(surname)
data_dum['Surname']=firstname

In [ ]:
data_dum=data_dum.drop(['Name','Age','Fare'],axis=1)

In [ ]:
X=data_dum.iloc[:891].values
y_test=data_dum.iloc[891:].values
y=survived_train.values
data_dum.head()


In [ ]:
alldied=[]
atleastone=[]
f=0
for i in data_dum['Surname']:
    count1=0
    count2=0
    for j,k in zip(X[:891,-1],range(891)):
        if i==j:
            if y[k]==0:
                count1=count1+1
            else:
                count2=count2+1
    if count1+count2>=3:
        if count1>=2:
            alldied.append(1)
        else:
            alldied.append(0)
        if count2>=1:
            atleastone.append(1)
        else:
            atleastone.append(0)
    else:
        alldied.append(0)
        atleastone.append(0)
data_dum['SurnameAllDied']=alldied
data_dum['SurnameOneLived']=atleastone

In [ ]:
data_dum.head()

In [ ]:
data_dum=data_dum.drop(['Surname'],axis=1)

In [ ]:
X=data_dum.iloc[:891].values
y_test=data_dum.iloc[891:].values

In [ ]:
X

In [ ]:
model = tree.DecisionTreeClassifier(max_depth=3, random_state=1)
model.fit(X, y)
predictions = model.predict(y_test)
output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
classifier = Sequential()
classifier.add(Dense(output_dim = 4, init = 'uniform', activation = 'relu', input_dim = 7))
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X,y, batch_size = 10, nb_epoch = 100)

In [ ]:
y_pred = classifier.predict(y_test)
predictions=[]
for i in y_pred:
    if i > 0.5:
        predictions.append(1)
    else:
        predictions.append(0)

In [ ]:
print(predictions)

In [ ]:
output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)